In [1]:
import numpy as np
import pandas as pd
import json
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier

In [3]:
with open('train.json', 'r') as f:
    datastore = json.load(f)

In [36]:
df = pd.DataFrame(datastore)
ings = df['ingredients']
print len(df), ings[0]

39774 [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']


In [5]:
ing_list = []
for ele in ings:
    for i in ele:
        if i not in ing_list:
            ing_list.append(i)

In [6]:
len(ing_list)

6714

In [7]:
train_df  = pd.DataFrame(0, index=np.arange(len(df)), columns=ing_list)

In [8]:
train_df.columns

Index([u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic',
       u'pepper', u'purple onion', u'seasoning', u'garbanzo beans',
       u'feta cheese crumbles', u'plain flour',
       ...
       u'Oscar Mayer Cotto Salami', u'Challenge Butter', u'orange glaze',
       u'cholesterol free egg substitute', u'ciabatta loaf',
       u'Lipton® Iced Tea Brew Family Size Tea Bags',
       u'Hidden Valley® Greek Yogurt Original Ranch® Dip Mix', u'lop chong',
       u'tomato garlic pasta sauce', u'crushed cheese crackers'],
      dtype='object', length=6714)

In [9]:
train_df.shape[0]

39774

In [10]:
train_df.iloc[0]

romaine lettuce                                        0
black olives                                           0
grape tomatoes                                         0
garlic                                                 0
pepper                                                 0
purple onion                                           0
seasoning                                              0
garbanzo beans                                         0
feta cheese crumbles                                   0
plain flour                                            0
ground pepper                                          0
salt                                                   0
tomatoes                                               0
ground black pepper                                    0
thyme                                                  0
eggs                                                   0
green tomatoes                                         0
yellow corn meal               

In [11]:
for i, row in df.iterrows():
    for ing in row['ingredients']:
        train_df.loc[i][ing] =1
        #print ing, new_df.loc[i][ing]
    

In [12]:
cui_list = df['cuisine'].values
train_df['cuisine'] = cui_list

In [13]:
id_list = df['id'].values
train_df['id'] = id_list

In [38]:
print train_df.loc[0]

romaine lettuce                                            1
black olives                                               1
grape tomatoes                                             1
garlic                                                     1
pepper                                                     1
purple onion                                               1
seasoning                                                  1
garbanzo beans                                             1
feta cheese crumbles                                       1
plain flour                                                0
ground pepper                                              0
salt                                                       0
tomatoes                                                   0
ground black pepper                                        0
thyme                                                      0
eggs                                                       0
green tomatoes          

In [14]:
feature_cols = ing_list
X = train_df.loc[:, feature_cols]

In [15]:
Y = train_df['cuisine']
target_cols = ['cuisine']

In [16]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [17]:
with open('test.json', 'r') as f:
    test_datastore = json.load(f)

In [18]:
df2 = pd.DataFrame(test_datastore)

In [19]:
test_df  = pd.DataFrame(0, index=np.arange(len(df2)), columns=ing_list)

In [20]:
for i, row in df2.iterrows():
    for ing in row['ingredients']:
        test_df.loc[i][ing] =1
        #print ing, new_df.loc[i][ing]

In [21]:
test_id_list = df2['id'].values
test_df['id'] = test_id_list

In [22]:
test_X = test_df.loc[:, feature_cols]

In [23]:
prediction = clf.predict(test_X)

In [24]:
results = pd.DataFrame(test_id_list, prediction)

In [25]:
results.to_csv('results.csv', sep=',', encoding='utf-8')

In [26]:
def get_train_test(df, y_col, x_cols, ratio):
    """ 
    This method transforms a dataframe into a train and test set, for this you need to specify:
    1. the ratio train : test (usually 0.7)
    2. the column with the Y_values
    """
    mask = np.random.rand(len(df)) < ratio
    df_train = df[mask]
    df_test = df[~mask]
    
    X_train = df_train.loc[:, x_cols].values
    X_test = df_test.loc[:, x_cols].values
    
    Y_train = df_train.loc[:, y_col].values
    Y_test = df_test.loc[:, y_col].values
    
    return df_train, df_test, X_train, Y_train, X_test, Y_test

In [27]:
train_test_ratio = 0.7
df_train, df_test, X_train, Y_train, X_test, Y_test = get_train_test(train_df, target_cols, feature_cols, train_test_ratio)

In [28]:
clf_dt = tree.DecisionTreeClassifier()
clf_dt = clf_dt.fit(X_train, Y_train)
dt_train_score = clf_dt.score(X_train, Y_train)
dt_test_score = clf_dt.score(X_test, Y_test)
print dt_train_score, dt_test_score

0.999784475017 0.604775869292


In [29]:
clf_nb = GaussianNB()
clf_nb = clf_nb.fit(X_train, Y_train.ravel())
nb_train_score = clf_nb.score(X_train, Y_train)
nb_test_score = clf_nb.score(X_test, Y_test)
print nb_train_score, nb_test_score

0.5444161069 0.383996648513


In [30]:
clf_rfc = RandomForestClassifier(n_estimators=1000)
clf_rfc = clf_rfc.fit(X_train, Y_train.ravel())
rfc_train_score = clf_rfc.score(X_train, Y_train)
rfc_test_score = clf_rfc.score(X_test, Y_test)
print rfc_train_score, rfc_test_score

0.999784475017 0.71193967323


In [31]:
clf_lr = LogisticRegression()
clf_lr = clf_lr.fit(X_train, Y_train.ravel())
lr_train_score = clf_lr.score(X_train, Y_train)
lr_test_score = clf_lr.score(X_test, Y_test)
print lr_train_score, lr_test_score

0.87603721398 0.780058651026


In [32]:
clf_mlp = MLPClassifier(alpha = 1)
clf_mlp = clf_mlp.fit(X_train, Y_train.ravel())
mlp_train_score = clf_mlp.score(X_train, Y_train)
mlp_test_score = clf_mlp.score(X_test, Y_test)
print mlp_train_score, mlp_test_score

0.726211430008 0.70297444491


In [33]:
C_param_range = [0.001,0.01,0.1,1,10,100]
for i in C_param_range:
    clf_lr = LogisticRegression(penalty = 'l2', C = i,random_state = 0)
    clf_lr = clf_lr.fit(X_train, Y_train.ravel())
    lr_train_score = clf_lr.score(X_train, Y_train)
    lr_test_score = clf_lr.score(X_test, Y_test)
    print i, lr_train_score, lr_test_score

0.001 0.481446891052 0.483200670297
0.01 0.636624878767 0.628906577294
0.1 0.77549480944 0.739673229996
1 0.87603721398 0.780058651026
10 0.939724846438 0.771260997067
100 0.961887998851 0.736908253037


In [34]:
n_estimators_range = [10, 100, 500]
criterion_range = ['gini', 'entropy']
for i in n_estimators_range:
    for j in criterion_range:
        clf_rfc = RandomForestClassifier(n_estimators=i, criterion=j)
        clf_rfc = clf_rfc.fit(X_train, Y_train.ravel())
        rfc_train_score = clf_rfc.score(X_train, Y_train)
        rfc_test_score = clf_rfc.score(X_test, Y_test)
        print i, j, rfc_train_score, rfc_test_score

10 gini 0.99195373397 0.663259321324
10 entropy 0.992708071411 0.627398408044
100 gini 0.999784475017 0.708001675744
100 entropy 0.999784475017 0.675994972769
500 gini 0.999784475017 0.715961457897
500 entropy 0.999784475017 0.684625052367


In [40]:
C_param_range = [2,3,4,5,6,7]
for i in C_param_range:
    clf_lr = LogisticRegression(penalty = 'l2', C = i,random_state = 0)
    clf_lr = clf_lr.fit(X_train, Y_train.ravel())
    lr_train_score = clf_lr.score(X_train, Y_train)
    lr_test_score = clf_lr.score(X_test, Y_test)
    print i, lr_train_score, lr_test_score

2 0.900176012069 0.782739840804
3 0.913790006825 0.782823627985
4 0.921405222889 0.780477586929
5 0.92661374331 0.778801843318
6 0.930600955494 0.776874738165
7 0.93394159273 0.775785504818


In [49]:
# selected algorithm = logistical regression with parameters C=1

clf_final = LogisticRegression(penalty = 'l2', C = 3,random_state = 0)
clf_final = clf_final.fit(X_train, Y_train.ravel())
fin_prediction = clf_final.predict(test_X)


In [51]:
output = {'id':test_id_list, 'cuisine':fin_prediction}
fin_results = pd.DataFrame(output)
fin_results.to_csv('output.csv', sep=',', encoding='utf-8', index=False, columns=['id','cuisine'], header=True)